In [0]:
# importando as bibliotecas necessárias
from pyspark.sql import functions as f
from pyspark.sql.window import Window
import datetime

In [0]:
dbutils.widgets.text("processing_date", "")
processing_date_str = dbutils.widgets.get("processing_date")
# processing_date_str = "2025-08-16" # usando a data atual como exemplo
processing_date = datetime.datetime.strptime(processing_date_str, "%Y-%m-%d").date()
print(f"Processando o dia: {processing_date}")

In [0]:
gold_payouts_table = "rp.gold.cdi_daily_payouts"

In [0]:
# Lendo a tabela da camada silver com saldos elegíveis
silver_balances_df = spark.table('rp.silver.daily_eligible_balances').filter(f.col("processing_date") == processing_date)
# Lendo a tabela de calendário.
calendar_df = spark.table('rp.bronze.dim_calendario') \
    .withColumn("date", f.to_date(f.col("data")))

In [0]:
# Calculando o número de dias corridos para creditar os juros.
days_to_accrue_df = silver_balances_df \
    .withColumn("days_to_accrue", f.datediff(f.col("processing_date"), f.col("last_business_day")))

In [0]:
# Simulando tabela de taxas de juros 
# esta tabela seria mantida e atualizada por uma equipe financeira.
interest_rates_data = [
    (datetime.date(2025, 8, 14), 0.1150), # quinta-feira
    (datetime.date(2025, 8, 15), 0.1155)  # sexta-feira
]
# Criando dataframe com a taxa de juros
interest_rates_df = spark.createDataFrame(interest_rates_data, ["rate_date", "annual_rate"]) \
    .withColumn("daily_rate", f.pow(f.lit(1) + f.col("annual_rate"), f.lit(1/252)) - 1)

In [0]:
# Capturando o último dia útil antes da data de processamento.
# usamos a coluna 'flag_dia_util_bancario' para identificar os dias úteis.
last_business_day_row = calendar_df \
    .filter((f.col("data") < processing_date) & (f.col("flag_dia_util_bancario") == "S")) \
    .orderBy(f.col("data").desc()) \
    .first()

if not last_business_day_row:
    dbutils.notebook.exit(f"nenhum dia útil encontrado antes de {processing_date_str}. verifique a tabela de calendário.")

last_business_day = last_business_day_row.data
print(f"último dia útil considerado para o cálculo: {last_business_day}")

In [0]:
# Obtendo a taxa de juros referente ao último dia útil.
interest_rate_for_day = interest_rates_df \
    .filter(f.col("rate_date") == last_business_day) \
    .select("daily_rate") \
    .first()

if not interest_rate_for_day:
    dbutils.notebook.exit(f"taxa de juros não encontrada para o último dia útil: {last_business_day}")

daily_rate = interest_rate_for_day.daily_rate

In [0]:
# calculando o pagamento final, aplicando a taxa diária e o número de dias.
gold_payout_df = days_to_accrue_df.withColumn(
    "interest_payout",
    f.round(
        f.col("balance_on_last_biz_day") * f.lit(daily_rate) * f.col("days_to_accrue"),
        4
    )
)

In [0]:
# Formatando a tabela final para a camada gold e filtrando apenas transações com valor a ser pago
final_gold_df = gold_payout_df.select(
    f.col("user_id"),
    f.col("processing_date").alias("payout_date"),
    f.col("balance_on_last_biz_day").alias("eligible_balance"),
    f.col("last_business_day"),
    f.col("days_to_accrue"),
    f.lit(daily_rate).alias("applied_daily_rate"),
    f.col("interest_payout")
).filter(f.col("interest_payout") > 0) # apenas transações com valor a ser pago

print("tabela final de pagamentos (camada gold):")

In [0]:
# Salvando a tabela final na camada gold.
(
    final_gold_df.write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"payout_date = '{processing_date_str}'")
    .partitionBy("payout_date")
    .saveAsTable(gold_payouts_table)
)

print(f"tabela de pagamentos salva com sucesso em '{gold_payouts_table}' para a data {processing_date_str}.")